In [133]:
import pandas as pd
import numpy as np
import catboost as cb

In [134]:
__get_next_month = {}

def get_next_month(date: str) -> str:
    if date in __get_next_month:
        return __get_next_month[date]
    else:
        next = __get_next_month[date] = str(pd.Period(date, freq="M") + 1)

        return next

## Cleanup the data

In [135]:
df_raw = pd.read_csv('history.csv')

df = df_raw.drop(columns=['ads_shown_current_day', 'ads_revenue_current_day'])

df = df.sort_values('id', ascending=True)

df = df.drop_duplicates(subset=['id', 'current_month'], keep='first')

df.reset_index(drop=True, inplace=True)

df = df.set_index(['id', 'current_month'], drop=False)

df.sort_index()

id  logins_current_month  country traffic_type  \
id     current_month                                                       
1      2024-07             1                    13       81         paid   
       2024-09             1                     1       81         paid   
2      2024-10             2                     1      193         paid   
3      2024-12             3                     7      180         paid   
       2025-01             3                     2      180         paid   
...                      ...                   ...      ...          ...   
133672 2025-01        133672                     2      173         paid   
133673 2024-10        133673                    14       49         paid   
133674 2024-07        133674                     2      208         paid   
       2024-11        133674                     3      208         paid   
       2025-01        133674                     1      208         paid   

                     platform registration_date  ads_shown_current_month  \
id     current_month                                                       
1      2024-07            iOS        2024-06-13                      100   
       2024-09            iOS        2024-06-13                        1   
2      2024-10        Android        2024-06-01                        0   
3      2024-12        Android        2024-12-12                        0   
       2025-01        Android        2024-12-12                        0   
...                       ...               ...                      ...   
133672 2025-01        Android        2024-12-26                        0   
133673 2024-10        Android        2024-10-11                       32   
133674 2024-07        Android        2024-05-08                        2   
       2024-11        Android        2024-05-08                        0   
       2025-01        Android        2024-05-08                        0   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
1      2024-07                        21.373946               23.33529   
       2024-09                         0.867439                8.87251   
2      2024-10                         0.000000                0.00000   
3      2024-12                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   
...                                         ...                    ...   
133672 2025-01                         0.000000                0.00000   
133673 2024-10                         0.042859                0.00000   
133674 2024-07                         0.137498                0.00000   
       2024-11                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   

                      games_pvp  ...  offers  currency_1  currency_2  \
id     current_month             ...                                   
1      2024-07              265  ...       0       37845         730   
       2024-09               18  ...       0           0           0   
2      2024-10                0  ...       0           0           0   
3      2024-12               26  ...       0           0           0   
       2025-01                1  ...       0           0           0   
...                         ...  ...     ...         ...         ...   
133672 2025-01                9  ...       0           0           0   
133673 2024-10              161  ...       0           0           0   
133674 2024-07                5  ...       0           0           0   
       2024-11                7  ...       0           0           0   
       2025-01                0  ...       0           0           0   

                      currency_3  currency_4  currency_5  currency_6  \
id     current_month                                                   
1   

In [136]:
# df[ df['current_month'] == '2025-04']

df

id  logins_current_month  country traffic_type  \
id     current_month                                                       
1      2024-07             1                    13       81         paid   
       2024-09             1                     1       81         paid   
2      2024-10             2                     1      193         paid   
3      2024-12             3                     7      180         paid   
       2025-01             3                     2      180         paid   
...                      ...                   ...      ...          ...   
133672 2025-01        133672                     2      173         paid   
133673 2024-10        133673                    14       49         paid   
133674 2025-01        133674                     1      208         paid   
       2024-07        133674                     2      208         paid   
       2024-11        133674                     3      208         paid   

                     platform registration_date  ads_shown_current_month  \
id     current_month                                                       
1      2024-07            iOS        2024-06-13                      100   
       2024-09            iOS        2024-06-13                        1   
2      2024-10        Android        2024-06-01                        0   
3      2024-12        Android        2024-12-12                        0   
       2025-01        Android        2024-12-12                        0   
...                       ...               ...                      ...   
133672 2025-01        Android        2024-12-26                        0   
133673 2024-10        Android        2024-10-11                       32   
133674 2025-01        Android        2024-05-08                        0   
       2024-07        Android        2024-05-08                        2   
       2024-11        Android        2024-05-08                        0   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
1      2024-07                        21.373946               23.33529   
       2024-09                         0.867439                8.87251   
2      2024-10                         0.000000                0.00000   
3      2024-12                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   
...                                         ...                    ...   
133672 2025-01                         0.000000                0.00000   
133673 2024-10                         0.042859                0.00000   
133674 2025-01                         0.000000                0.00000   
       2024-07                         0.137498                0.00000   
       2024-11                         0.000000                0.00000   

                      games_pvp  ...  offers  currency_1  currency_2  \
id     current_month             ...                                   
1      2024-07              265  ...       0       37845         730   
       2024-09               18  ...       0           0           0   
2      2024-10                0  ...       0           0           0   
3      2024-12               26  ...       0           0           0   
       2025-01                1  ...       0           0           0   
...                         ...  ...     ...         ...         ...   
133672 2025-01                9  ...       0           0           0   
133673 2024-10              161  ...       0           0           0   
133674 2025-01                0  ...       0           0           0   
       2024-07                5  ...       0           0           0   
       2024-11                7  ...       0           0           0   

                      currency_3  currency_4  currency_5  currency_6  \
id     current_month                                                   
1   

In [137]:
# # check multiplied entries (same id + current_month)
#
# grouped = df.groupby([ 'id', 'current_month' ], as_index=False).agg(ids = ('id', 'count'))
#
# grouped[ grouped[ 'ids' ] > 1 ]

In [138]:
# add training data
def ads_revenue_next_month(row):
    key = (row['id'], get_next_month(row[ 'current_month' ]))
    if key in df.index:
        next_row = df.loc[ key ]

        return next_row[ 'ads_revenue_current_month' ]
    else:
        return 0

def revenue_next_month(row):
    key = (row['id'], get_next_month(row[ 'current_month' ]))
    if key in df.index:
        next_row = df.loc[ key ]

        return next_row[ 'revenue_current_month' ]
    else:
        return 0

def is_active_next_month(row):
    key = (row['id'], get_next_month(row[ 'current_month' ]))
    if key in df.index:
        next_row = df.loc[ key ]

        return next_row[ 'logins_current_month' ] > 0
    else :
        return False

df[ 'ads_revenue_next_month' ] = df.apply(ads_revenue_next_month, axis=1)
df[ 'revenue_next_month' ] = df.apply(revenue_next_month, axis=1)
# df[ 'log_ads_revenue_next_month' ] = np.log1p(df[ 'ads_revenue_next_month' ])
# df[ 'log_revenue_next_month' ] = np.log1p(df[ 'revenue_next_month' ])
df[ 'is_active_next_month' ] = df.apply(is_active_next_month, axis=1)

In [139]:
df[ 'months_after_reg' ] = df.apply(lambda row: (pd.Period(row[ 'current_month' ], freq="M") - pd.Period(row[ 'registration_date' ], freq="M")).n, axis=1)
df.drop(columns='registration_date', inplace=True)

In [140]:
df

id  logins_current_month  country traffic_type  \
id     current_month                                                       
1      2024-07             1                    13       81         paid   
       2024-09             1                     1       81         paid   
2      2024-10             2                     1      193         paid   
3      2024-12             3                     7      180         paid   
       2025-01             3                     2      180         paid   
...                      ...                   ...      ...          ...   
133672 2025-01        133672                     2      173         paid   
133673 2024-10        133673                    14       49         paid   
133674 2025-01        133674                     1      208         paid   
       2024-07        133674                     2      208         paid   
       2024-11        133674                     3      208         paid   

                     platform  ads_shown_current_month  \
id     current_month                                     
1      2024-07            iOS                      100   
       2024-09            iOS                        1   
2      2024-10        Android                        0   
3      2024-12        Android                        0   
       2025-01        Android                        0   
...                       ...                      ...   
133672 2025-01        Android                        0   
133673 2024-10        Android                       32   
133674 2025-01        Android                        0   
       2024-07        Android                        2   
       2024-11        Android                        0   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
1      2024-07                        21.373946               23.33529   
       2024-09                         0.867439                8.87251   
2      2024-10                         0.000000                0.00000   
3      2024-12                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   
...                                         ...                    ...   
133672 2025-01                         0.000000                0.00000   
133673 2024-10                         0.042859                0.00000   
133674 2025-01                         0.000000                0.00000   
       2024-07                         0.137498                0.00000   
       2024-11                         0.000000                0.00000   

                      games_pvp  wins_pvp  ...  currency_4  currency_5  \
id     current_month                       ...                           
1      2024-07              265       205  ...          26           0   
       2024-09               18         7  ...           0           0   
2      2024-10                0         0  ...           0           0   
3      2024-12               26        20  ...           0           0   
       2025-01                1         0  ...           0           0   
...                         ...       ...  ...         ...         ...   
133672 2025-01                9         2  ...           0           0   
133673 2024-10              161        90  ...           0           0   
133674 2025-01                0         0  ...           0           0   
       2024-07                5         1  ...           0           0   
       2024-11                7         1  ...           0           0   

                      currency_6  currency_7  current_avg_ping  current_month  \
id     current_month                                                            
1      2024-07                 0           0          0.275935        2024-07   
       2024-09                 0           0          0.263144        2024-09   
2      2024-10                 0

# Чистка данных

In [141]:
def outliers_robust_z(s: pd.Series, k: float = 3.5) -> pd.Index:
    m = s.median(skipna=True)
    mad = (s - m).abs().median(skipna=True)                       # median{|x_t - m|}
    if mad == 0 or not np.isfinite(mad):
        return pd.Index([])
    sigma_hat = 1.4826 * mad
    z = (s - m) / sigma_hat
    return s.index[z.abs() > k]

In [142]:
df = df.copy()

# df['log_revenue_current_month'] = np.log1p(df['revenue_current_month'])
# df['log_ads_revenue_current_month'] = np.log1p(df['ads_revenue_current_month'])


In [143]:
df[ df['current_month'] == '2025-04' ].to_csv('predict.csv')

df = df[ df['current_month'] != '2025-04' ]

In [144]:
has_revenue_next_month = df[df['revenue_next_month'] > 0]

has_revenue_next_month

,,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,currency_4,currency_5,currency_6,currency_7,current_avg_ping,current_month,ads_revenue_next_month,revenue_next_month,is_active_next_month,months_after_reg
id,current_month,,,,,,,,,,,,,,,,,,,,,
10,2025-03,10,13,164,paid,iOS,247,13.075025,55.007869,314,191,...,6,90,3350,1550,0.041453,2025-03,26.512756,121.358247,True,0
59,2024-09,59,3,50,organic,iOS,7,0.072389,0.000000,34,25,...,0,0,0,0,0.267038,2024-09,0.336345,0.580000,True,0
78,2024-07,78,31,153,paid,Android,75,1.055468,0.000000,187,107,...,5,0,0,0,0.408551,2024-07,1.619926,1.386000,True,3
211,2024-07,211,29,106,paid,Android,65,1.056081,1.393000,92,73,...,10,0,0,0,0.555761,2024-07,0.330513,0.693000,True,5
244,2024-10,244,7,180,paid,Android,21,0.670987,0.000000,144,93,...,0,0,0,0,0.319014,2024-10,0.654616,0.693000,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133485,2024-09,133485,10,71,paid,Android,109,4.776956,0.000000,226,128,...,0,0,0,0,0.351959,2024-09,13.794182,3.479000,True,0
133497,2025-03,133497,2,112,paid,iOS,32,1.479628,3.222924,82,53,...,0,0,0,0,0.298862,2025-03,15.251650,19.098080,True,0
133538,2024-08,133538,11,112,organic,iOS,11,3.949728,6.532887,104,74,...,0,0,0,0,0.324780,2024-08,2.066304,107.834951,True,0


In [145]:
outliers_rev_next_month = outliers_robust_z(has_revenue_next_month['revenue_current_month'], k=3)
outliers_rev_next_month

MultiIndex([(    10, '2025-03'),
            (   358, '2024-11'),
            (   358, '2024-10'),
            (   358, '2024-09'),
            (   421, '2025-03'),
            (   421, '2025-02'),
            (   502, '2024-07'),
            (   502, '2024-10'),
            (   502, '2024-09'),
            (   507, '2024-07'),
            ...
            (133082, '2025-01'),
            (133082, '2024-07'),
            (133082, '2024-11'),
            (133082, '2025-02'),
            (133082, '2024-10'),
            (133117, '2024-11'),
            (133154, '2025-01'),
            (133233, '2024-12'),
            (133233, '2024-11'),
            (133538, '2024-09')],
           names=['id', 'current_month'], length=2337)

In [146]:
clean1 = df #.drop(outliers_rev_next_month)

clean1

id  logins_current_month  country traffic_type  \
id     current_month                                                       
1      2024-07             1                    13       81         paid   
       2024-09             1                     1       81         paid   
2      2024-10             2                     1      193         paid   
3      2024-12             3                     7      180         paid   
       2025-01             3                     2      180         paid   
...                      ...                   ...      ...          ...   
133672 2025-01        133672                     2      173         paid   
133673 2024-10        133673                    14       49         paid   
133674 2025-01        133674                     1      208         paid   
       2024-07        133674                     2      208         paid   
       2024-11        133674                     3      208         paid   

                     platform  ads_shown_current_month  \
id     current_month                                     
1      2024-07            iOS                      100   
       2024-09            iOS                        1   
2      2024-10        Android                        0   
3      2024-12        Android                        0   
       2025-01        Android                        0   
...                       ...                      ...   
133672 2025-01        Android                        0   
133673 2024-10        Android                       32   
133674 2025-01        Android                        0   
       2024-07        Android                        2   
       2024-11        Android                        0   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
1      2024-07                        21.373946               23.33529   
       2024-09                         0.867439                8.87251   
2      2024-10                         0.000000                0.00000   
3      2024-12                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   
...                                         ...                    ...   
133672 2025-01                         0.000000                0.00000   
133673 2024-10                         0.042859                0.00000   
133674 2025-01                         0.000000                0.00000   
       2024-07                         0.137498                0.00000   
       2024-11                         0.000000                0.00000   

                      games_pvp  wins_pvp  ...  currency_4  currency_5  \
id     current_month                       ...                           
1      2024-07              265       205  ...          26           0   
       2024-09               18         7  ...           0           0   
2      2024-10                0         0  ...           0           0   
3      2024-12               26        20  ...           0           0   
       2025-01                1         0  ...           0           0   
...                         ...       ...  ...         ...         ...   
133672 2025-01                9         2  ...           0           0   
133673 2024-10              161        90  ...           0           0   
133674 2025-01                0         0  ...           0           0   
       2024-07                5         1  ...           0           0   
       2024-11                7         1  ...           0           0   

                      currency_6  currency_7  current_avg_ping  current_month  \
id     current_month                                                            
1      2024-07                 0           0          0.275935        2024-07   
       2024-09                 0           0          0.263144        2024-09   
2      2024-10                 0

In [147]:
has_ads_revenue_next_month = clean1[clean1['ads_revenue_next_month'] > 0]
has_ads_revenue_next_month

id  logins_current_month  country traffic_type  \
id     current_month                                                       
4      2024-12             4                    22      106         paid   
6      2025-01             6                    11      133      organic   
       2025-02             6                    27      133      organic   
       2025-03             6                    31      133      organic   
8      2024-12             8                     6       46         paid   
...                      ...                   ...      ...          ...   
133649 2025-01        133649                    11      153      organic   
133658 2024-07        133658                    24       86         paid   
133659 2024-07        133659                    12      153         paid   
133661 2024-12        133661                    10      112      organic   
133670 2025-01        133670                     7       81         paid   

                     platform  ads_shown_current_month  \
id     current_month                                     
4      2024-12        Android                       11   
6      2025-01        Android                       47   
       2025-02        Android                      146   
       2025-03        Android                      163   
8      2024-12        Android                      112   
...                       ...                      ...   
133649 2025-01        Android                       26   
133658 2024-07        Android                     1067   
133659 2024-07        Android                       28   
133661 2024-12            iOS                       52   
133670 2025-01            iOS                       31   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
4      2024-12                         0.077312                  0.000   
6      2025-01                         0.000000                  0.000   
       2025-02                         0.036284                  0.000   
       2025-03                         0.106053                  0.000   
8      2024-12                         0.108613                  0.000   
...                                         ...                    ...   
133649 2025-01                         0.068609                  0.000   
133658 2024-07                         3.436036                  1.386   
133659 2024-07                         0.015844                  0.000   
133661 2024-12                         1.721284                  0.000   
133670 2025-01                         2.174221                  0.000   

                      games_pvp  wins_pvp  ...  currency_4  currency_5  \
id     current_month                       ...                           
4      2024-12              266       140  ...           0           0   
6      2025-01              145        71  ...           0           0   
       2025-02              385       186  ...           0           0   
       2025-03              246       125  ...           0           0   
8      2024-12              173       103  ...           0           0   
...                         ...       ...  ...         ...         ...   
133649 2025-01               62        41  ...           0           0   
133658 2024-07              728       485  ...          12           0   
133659 2024-07              120        68  ...           0           0   
133661 2024-12               90        54  ...           0           0   
133670 2025-01               51        34  ...           0           0   

                      currency_6  currency_7  current_avg_ping  current_month  \
id     current_month                                                            
4      2024-12                 0           0          0.396413        2024-12   
6      2025-01                 0           0          0.345713        2025-01   
       2025-02             13900

In [148]:
outliers_ads_rev_next_month = outliers_robust_z(has_ads_revenue_next_month['ads_revenue_next_month'], k=3.5)

outliers_ads_rev_next_month

MultiIndex([(    10, '2025-03'),
            (    78, '2024-08'),
            (   276, '2025-03'),
            (   276, '2025-02'),
            (   293, '2024-11'),
            (   293, '2024-12'),
            (   300, '2025-03'),
            (   311, '2024-07'),
            (   315, '2024-11'),
            (   348, '2024-08'),
            ...
            (133493, '2024-12'),
            (133497, '2025-03'),
            (133538, '2024-08'),
            (133553, '2025-03'),
            (133555, '2024-08'),
            (133555, '2024-10'),
            (133555, '2024-09'),
            (133580, '2024-07'),
            (133584, '2024-09'),
            (133591, '2024-10')],
           names=['id', 'current_month'], length=11245)

In [149]:
clean2 = clean1 #.drop(outliers_ads_rev_next_month)

In [150]:
clean2

id  logins_current_month  country traffic_type  \
id     current_month                                                       
1      2024-07             1                    13       81         paid   
       2024-09             1                     1       81         paid   
2      2024-10             2                     1      193         paid   
3      2024-12             3                     7      180         paid   
       2025-01             3                     2      180         paid   
...                      ...                   ...      ...          ...   
133672 2025-01        133672                     2      173         paid   
133673 2024-10        133673                    14       49         paid   
133674 2025-01        133674                     1      208         paid   
       2024-07        133674                     2      208         paid   
       2024-11        133674                     3      208         paid   

                     platform  ads_shown_current_month  \
id     current_month                                     
1      2024-07            iOS                      100   
       2024-09            iOS                        1   
2      2024-10        Android                        0   
3      2024-12        Android                        0   
       2025-01        Android                        0   
...                       ...                      ...   
133672 2025-01        Android                        0   
133673 2024-10        Android                       32   
133674 2025-01        Android                        0   
       2024-07        Android                        2   
       2024-11        Android                        0   

                      ads_revenue_current_month  revenue_current_month  \
id     current_month                                                     
1      2024-07                        21.373946               23.33529   
       2024-09                         0.867439                8.87251   
2      2024-10                         0.000000                0.00000   
3      2024-12                         0.000000                0.00000   
       2025-01                         0.000000                0.00000   
...                                         ...                    ...   
133672 2025-01                         0.000000                0.00000   
133673 2024-10                         0.042859                0.00000   
133674 2025-01                         0.000000                0.00000   
       2024-07                         0.137498                0.00000   
       2024-11                         0.000000                0.00000   

                      games_pvp  wins_pvp  ...  currency_4  currency_5  \
id     current_month                       ...                           
1      2024-07              265       205  ...          26           0   
       2024-09               18         7  ...           0           0   
2      2024-10                0         0  ...           0           0   
3      2024-12               26        20  ...           0           0   
       2025-01                1         0  ...           0           0   
...                         ...       ...  ...         ...         ...   
133672 2025-01                9         2  ...           0           0   
133673 2024-10              161        90  ...           0           0   
133674 2025-01                0         0  ...           0           0   
       2024-07                5         1  ...           0           0   
       2024-11                7         1  ...           0           0   

                      currency_6  currency_7  current_avg_ping  current_month  \
id     current_month                                                            
1      2024-07                 0           0          0.275935        2024-07   
       2024-09                 0           0          0.263144        2024-09   
2      2024-10                 0

In [151]:
clean2.to_csv('training.csv', index=False)

In [152]:
df = clean2

agg_df = df.groupby('country', as_index=False).agg(
    log_revenue_next_month = ('log_revenue_next_month', 'sum'),
    log_revenue_next_month_count = ('log_revenue_next_month', 'count')
)

agg_df

KeyError: "Column(s) ['log_revenue_next_month'] do not exist"